In [2]:
import pandas as pd

In [3]:
data = [
  {
    "id": 1,
    "name": {
      "first": "Coleen",
      "last": "Volk"
    }
  },
  {
    "name": {
      "given": "Mark",
      "family": "Regner"
    }
  },
  {
    "id": 2,
    "name": "Faye Raker"
  },
  
]
pd.json_normalize(data)

,id,name.first,name.last,name.given,name.family,name
0,1.0,Coleen,Volk,NaN,NaN,NaN
1,NaN,NaN,NaN,Mark,Regner,NaN
2,2.0,NaN,NaN,NaN,NaN,Faye Raker
